# None Such Thing: Listening Habits
### Do all Kevin Nguyens listen to Illenium, or am I just racist?

If you grew up part of the Asian-American generation Z like me, there's a good chance you're aware of the stereotype of _Kevin Nguyen_. He drives a Civic, probably juuls, definitely listens to Illenium and pretends to like Travis Scott. But how true is this? Let's find out! 

#### The Game Plan

Let's set the ground rules before we dive in. To ascertain this information, we're going to need address the first part of our question: Do all Kevin Nguyens listen to Illenium? This is kind of tough. Spotify doesn't give us external access to a user's `liked` songs or listening history, so we'll use what we can get. We're going to aggregate _all_ the songs that each Kevin Nguyen has on _all_ his playlists. After all, if a song is on a playlist, he probably likes it in some way, even if it is ironically. But then again, who likes Illenium unironically? Joking.

Most of this will be done with the aid of `spotipy`, a lightweight Python wrapper for the Spotify Web API.

The initial setup is kind of tedious. To use the Client Credientials Flow, You need to register as a Spotify developer and obtain a `client id` and a `client secret`. This allows a higher throughput of requests that can be put to Spotify than the regular Authorization Code Flow. Not that we really need it. But this is actually registered as a Spotify app. Isn't life rich?

In [5]:
# set up the client id and secret
import os
os.environ["SPOTIPY_CLIENT_ID"] = "f09ddc90842441e68107eb66ef178403"
os.environ["SPOTIPY_CLIENT_SECRET"] = "316d709c188f48ac9d9152fed07f4de7"


Yes, I made both public on this notebook. No, I don't care.

In [9]:
# import the libraries and start the real work
import spotipy
from spotipy.oauth2 import SpotifyOAuth

auth_manager = SpotifyClientCredentials()
client = spotipy.Spotify(auth_manager=auth_manager)

### 1. Getting Comfortable With Spotipy

To get our feet wet, let's find out all of Illenium's albums and his top 10 songs.

In [24]:
illenium = 'spotify:artist:45eNHdiiabvmbp4erw26rg'
albums = client.artist_albums(illenium, album_type='album')['items']
songs = client.artist_top_tracks(illenium)['tracks']

In [25]:
for album in albums:
    print(album['name'])

ASCEND (Remixes)
ASCEND
Awake (Remixes)
Awake
Ashes (Remixes)
Ashes


In [30]:
for song in songs:
    print(song['name'])

Takeaway
In Your Arms (with X Ambassadors)
Nightlight
Feel Good (feat. Daya)
Good Things Fall Apart (with Jon Bellion)
Don't Let Me Down (feat. Daya) - Illenium Remix
Good Things Fall Apart (with Jon Bellion) [Tiësto's Big Room Remix]
Feel Something (With I Prevail)
Crashing (feat. Bahari)
Without Me - ILLENIUM Remix


Why is the Tiësto remix here. Gross. The original is so much better. Anyways, I digress. Now that we've got the hang of working with artists, lets get to the meat: profiles.

### 2. Working With Profiles

A quick search in the web player for "Kevin Nguyen" is a good starting point.

![title](pictures/nguyens.png)
<center>Wow. That is a lot of Kevin Nguyens. The full list actually goes down for miles, my lord.

Let's work with the first guy. Nothing personal, kid.

In [37]:
user_id = 'kevin.nguyen9852'
user = sp.user(user_id)
print(user.keys())

dict_keys(['display_name', 'external_urls', 'followers', 'href', 'id', 'images', 'type', 'uri'])


Huh. So on first glance, it seems like Spotify doesn't allow us explicit access to their playlists.

We still have access to all the urls, though, so let's just play it by ear for the moment. Clicking through, we find what is probably the most Kevin Nguyen set of playlists to have ever Kevin Nguyened.

![title](pictures/glock_brian.png)

Rich Brian with the strap is the best thing on this page. Let's take a look at who he's bumping.

In [101]:
# define a function that extracts all the artists from a playlist
from collections import Counter

def get_artists(playlist_id: str) -> Counter:
    playlist = client.playlist(playlist_id=playlist_id)
    tracks = playlist['tracks']['items']
    artists = Counter()
    for track in tracks:
        artists.update(artist['name'] for artist in track['track']['artists'])
    return artists

In [103]:
mitre_10_id = '5EbNNdQvJ8U4TyuqetE5pY?si=wHISzC8XTBOIbsJ0oRB7yQ'
get_artists(mitre_10_id)

Counter({'Kid Trunks': 5,
         'Bass Santana': 11,
         'Kin$oul': 6,
         'XXXTENTACION': 8,
         'Cooliecut': 1,
         'Flyboy Taratino': 1,
         'Smokepurpp': 9,
         'Rich Brian': 1,
         'Keith Ape': 2,
         'Travis Scott': 1,
         'Fat Nick': 3,
         'Pouya': 12,
         '$uicideBoy$': 43,
         'Sir Michael Rocks': 1,
         'Robb Bank$': 3,
         'Lil Pump': 2,
         'Lil Xan': 4,
         'Ski Mask The Slump God': 7,
         'Shakewell': 5,
         'Flyboy Tarantino': 2,
         'Killstation': 2,
         'ikabodVEINS': 1,
         'Lxui Savage': 2,
         'Ghostemane': 2,
         'Nefariou$': 1,
         'Craig Xen': 1,
         'Maxo Cream': 1,
         'Germ': 4,
         'RAMIREZ': 2,
         'Demotus': 2,
         'Kirb La Goop': 1,
         'Lil Peep': 2,
         'Lil Tracy': 1,
         'Trippie Redd': 3,
         '6ix9ine': 1,
         'Erick the Architect': 1,
         'Trippy tha Kid': 1,
         'hollow

Wow. This dude loves SuicideBoyS. No judgement, though.

### 3. Scraping the Barrel

So we have a problem. Spotify doesn't explicitly link a user and his playlists, which means we'll have to manually get the playlists for each user by using a scraper.